In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Data preprocessing
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10 dataset
# 32 x 32 x 3 images, 10 classes
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                      download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                     download=True, transform=transform)
testloader = DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

# Define CNN
# Architecture: 32x32 -> 16x
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.fc_layers = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(4*4*256, 1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, verbose=True)

def train(epochs):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(trainloader):
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            if i % 100 == 99:
                print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}')
                running_loss = 0.0
        
        # Validation
        val_loss = evaluate()
        scheduler.step(val_loss)

def evaluate():
    model.eval()
    correct = 0
    total = 0
    val_loss = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    accuracy = 100. * correct / total
    print(f'Accuracy: {accuracy:.2f}%')
    return val_loss / len(testloader)

100%|██████████| 170498071/170498071 [00:13<00:00, 12301442.08it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


/home/virtanej/.conda/envs/deeplearn/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [5]:
train(10)

[1, 100] loss: 1.916
[1, 200] loss: 1.549
[1, 300] loss: 1.378
Accuracy: 57.51%
[2, 100] loss: 1.149
[2, 200] loss: 1.076
[2, 300] loss: 1.007
Accuracy: 58.30%
[3, 100] loss: 0.921
[3, 200] loss: 0.865
[3, 300] loss: 0.841
Accuracy: 69.83%
[4, 100] loss: 0.779
[4, 200] loss: 0.769
[4, 300] loss: 0.740
Accuracy: 72.08%
[5, 100] loss: 0.700
[5, 200] loss: 0.694
[5, 300] loss: 0.683
Accuracy: 76.06%
[6, 100] loss: 0.630
[6, 200] loss: 0.630
[6, 300] loss: 0.620
Accuracy: 77.48%
[7, 100] loss: 0.585
[7, 200] loss: 0.581
[7, 300] loss: 0.577
Accuracy: 80.36%
[8, 100] loss: 0.543
[8, 200] loss: 0.555
[8, 300] loss: 0.529
Accuracy: 78.01%
[9, 100] loss: 0.534
[9, 200] loss: 0.502
[9, 300] loss: 0.514
Accuracy: 80.20%
[10, 100] loss: 0.481
[10, 200] loss: 0.494
[10, 300] loss: 0.474
Accuracy: 81.75%


In [6]:
def save_model(model, path='cifar10_cnn.pth'):
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")

def load_model(path='cifar10_cnn.pth'):
    model = CNN().to(device)
    model.load_state_dict(torch.load(path))
    model.eval()
    return model

In [7]:
save_model(model)

Model saved to cifar10_cnn.pth


In [ ]:
# Load and use the model
loaded_model = load_model()
test_input = next(iter(testloader))[0].to(device)
with torch.no_grad():
    prediction = loaded_model(test_input)